In [1]:
with open(file='/kaggle/input/melville-moby-dick/melville-moby_dick.txt', mode='r', encoding='utf-8') as input_fp:
    lines = input_fp.readlines()
print(len(lines))

22423


In [2]:
import pandas as pd
starts = [index for index, line in enumerate(lines) if line.startswith('CHAPTER')]
documents = []
for index, start in enumerate(starts):
    if index != len(starts)-1:
        end = starts[index+1]
    else:
        end = -1
    document = ' '.join(lines[start:end])
    documents.append(document)
document_df = pd.DataFrame(data={'chapter': documents})
document_df['length (word)'] = document_df['chapter'].apply(func=lambda x: len(x.split()))
document_df['length (char)'] = document_df['chapter'].str.len()

document_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   chapter        135 non-null    object
 1   length (word)  135 non-null    int64 
 2   length (char)  135 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 3.3+ KB


In [3]:
from plotly.express import scatter
scatter(data_frame=document_df, y='length (char)', trendline='lowess', trendline_color_override='orange', ).show()
scatter(data_frame=document_df, y='length (word)', trendline='lowess', trendline_color_override='orange', ).show()
scatter(data_frame=document_df, x='length (word)',  y='length (char)', hover_name=document_df.index.tolist(),
        trendline='lowess', trendline_color_override='orange',
        log_x=True, log_y=True).show()

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

counts = []
for document in documents:
    model = CountVectorizer(encoding='utf-8', stop_words='english', min_df=1, max_df=1.0, lowercase=True, ngram_range=(1, 1))
    count_result = model.fit_transform(raw_documents=document.split('\n'))
    names = model.get_feature_names_out().tolist()
    occurences = count_result.toarray().sum(axis=0).tolist()
    counts.append(pd.Series(data={name: occurences[index] for index, name in enumerate(names)}))

count_df = pd.DataFrame(data=counts).fillna(value=0)
count_df.head()

,abandon,abominate,absent,account,act,activity,affghanistan,afternoon,ago,ah,...,upheaved,veer,ventilated,voiced,voicelessly,vortex,weeps,whelmings,wink,writhed
0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
total_df = count_df.sum(axis=0).to_frame().reset_index()
total_df = total_df[total_df[0] > 30]
total_df.shape

(474, 2)

In [6]:
 count_df.idxmax(axis=1)

0             sea
1             old
2      harpooneer
3        queequeg
4            good
          ...    
130            ye
131           old
132         whale
133          ahab
134          ahab
Length: 135, dtype: object

In [7]:
 count_df.max(axis=1)

0      13.0
1       8.0
2      35.0
3      13.0
4       6.0
       ... 
130     5.0
131    14.0
132    28.0
133    27.0
134    37.0
Length: 135, dtype: float64